<a href="https://colab.research.google.com/github/Pugacheva-Julia/data-science/blob/main/Public_procurements_of_salt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Анализ закупок соли по данным, полученным с сайта http://zakupki.gov.ru/**


На работе мне поставили задачу проанализировать государственные закупки соли за 2019 год. В нашей компании принято работать с данными в Excel, составлять таблицы, затем рисовать графики и редактировать их в Power Point. 
Мне стало интересно провести исследование с использованием средств Python и представить результат в более удобном формате.

In [ ]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Импорт данных
df = pd.read_excel('salt.xlsx')
df.head(2)

,Номер реестровой записи контракта,Заказчик: наименование,регион,Способ размещения заказа,Контракт: дата,Контракт: номер,"Объект закупки: наименование товаров, работ, услуг",Код,Цена,"Вес, кг","Информация о поставщиках (исполнителях, подрядчиках) по контракту: наименование юридического лица (ф.и.о. физического лица)"
0,1702003579819000035',"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ИСПРАВИТЕЛЬНА...",70.0,Закупка у единственного поставщика (подрядчика...,2019-06-25,'120/19',Соль пищевая поваренная йодированная,10.84.30.130,5.0,224.47,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОМБИ"""
1,'1702003579819000046',"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""ИСПРАВИТЕЛЬНА...",70.0,Закупка у единственного поставщика (подрядчика...,2019-08-12,'156/19',Соль пищевая йодированная,10.84.30.130-00000002,5.0,468.00,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОМБИ"""


In [ ]:
# Переименование столбцов для удобства работы
df.columns = ['number', 'customer', 'region', 'method', 'date', 'contruct_id', 'product', 'code', 'price', 'volume', 'executor']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2928 entries, 0 to 2927
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   number       2928 non-null   object        
 1   customer     2928 non-null   object        
 2   region       2927 non-null   float64       
 3   method       2928 non-null   object        
 4   date         2928 non-null   datetime64[ns]
 5   contruct_id  2904 non-null   object        
 6   product      2928 non-null   object        
 7   code         2928 non-null   object        
 8   price        2922 non-null   float64       
 9   volume       2928 non-null   float64       
 10  executor     2928 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 251.8+ KB


In [ ]:
df['code'].unique().tolist()

# Код соли 10.84.30.130
# В выгрузку ошибочно попали данные по другим кодам

['10.84.30.130',
 '10.84.30.130-00000002',
 '10.84.30.140',
 '10.84.30.130-00000010',
 '10.84.30.110',
 '10.84.30.130-00000004',
 '10.84.30.130-00000009',
 '10.84.30.120',
 '10.84.30.130-00000011',
 '10.84.30.130-00000012',
 '10.84.30.130-00000005',
 '10.84.30.130-00000006',
 '10.84.30.000-00000007',
 '10.84.30.130-00000003',
 '10.84.30.130-00000007',
 '10.84.30.000-00000001',
 '10.84.30.130-00000008',
 '10.89.19.290',
 '10.84.30.000-00000002']

In [ ]:
# Удаление нерелевантных данных
df = df[df['code'] == '10.84.30.130']

# Также в выгрузку попадают данные не только за 2019 год, нужно их удалить
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.to_period('Y')
df = df[df['year'] == '2019']

**1.** Во-первых, я проанализировала способы размещения заказа.

In [ ]:
df_methods = df.groupby('method').agg({'number' : 'nunique'}).reset_index()
df_methods.sort_values('number', ascending=False)

,method,number
7,Электронный аукцион,459
3,Конкурс c ограниченным участием,55
0,Закупка у единственного поставщика (подрядчика...,53
2,Запрос котировок в электронной форме,20
1,Запрос котировок,4
4,Конкурс с ограниченным участием в электронной ...,4
5,Открытый конкурс в электронной форме,2
6,"Способ определения поставщика (подрядчика, исп...",1


In [ ]:
fig = px.pie(df_methods, values='number', names='method', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

Чаще всего для закупки соли проводят электронный аукцион. Следующими по популярности способами размещения заказа являются конкурс с ограниченным участием и закупка у единственного поставщика.


**2.** Во-вторых, нужно определить общую стоимость заказа, умножив цену за килограмм на объем закупки.

In [ ]:
df['value'] = df['price'] * df['volume']
df.sort_values('volume', ascending=False).head()

,number,customer,region,method,date,contruct_id,product,code,price,volume,executor,year,value
67,'1526204221519000179',ГЛАВНОЕ УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕН...,52.0,Электронный аукцион,2019-06-18,'03321000018190001570001/290',Соль,10.84.30.130,7.02,80000.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НИЖЕ...",2019,561600.0
52,'1165400456619000110',УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,16.0,Электронный аукцион,2019-06-27,'56/36-19/39/197-19',"Соль пищевая ГОСТ Р 51574-2018 высшего сорта, ...",10.84.30.130,6.97,68000.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИВИ""",2019,473960.0
2546,'1772237786619000327',ФЕДЕРАЛЬНАЯ СЛУЖБА ВОЙСК НАЦИОНАЛЬНОЙ ГВАРДИИ ...,77.0,Закупка у единственного поставщика (подрядчика...,2019-07-16,'0195400000219000415',Соль поваренная пищевая (с добавлением йода),10.84.30.130,18.00,38500.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «МЯСО...,2019,693000.0
411,'1673001687919000118',УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,67.0,Запрос котировок в электронной форме,2019-05-17,'223',"Соль поваренная пищевая, не ниже 1 сорта ГОСТ...",10.84.30.130,8.46,33000.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ХИРО...",2019,279180.0
1110,'1772237786619000325',ФЕДЕРАЛЬНАЯ СЛУЖБА ВОЙСК НАЦИОНАЛЬНОЙ ГВАРДИИ ...,77.0,Закупка у единственного поставщика (подрядчика...,2019-07-16,'0195400000219000414',Соль поваренная пищевая (с добавлением йода),10.84.30.130,10.40,17500.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «МЯСО...,2019,182000.0


Можно сдедать вывод, что самые больше закупки осуществляются федеральными службами исполнения наказаний и войск национальной гвардии.

**3.** В-третьих, целесообразно выявить корреляцию цены и даты контракты, чтобы оценить сезонность продукта.

In [ ]:
fig = px.scatter(df, x='date', y='price', hover_data=['volume'], color='volume', title='Сезонность продукта')
fig.show()

На графике видно, что цена соли не меняется в зависимости от даты. Следовательно, продукт не подвержен сезонным колебаниям.

**4.** В-четвертых, интересно распределение количества закупока по регионам.

In [ ]:
df_reg = df.groupby('region').agg({'number':'count', 'price':'mean'}).reset_index()
df_reg['region'] = df_reg['region'].astype(int)
df_reg['region'] = df_reg['region'].astype(str)
df_reg.head()

,region,number,price
0,1,6,15.736111
1,2,6,10.565000
2,4,1,9.500000
3,5,4,9.537500
4,7,12,15.394167


В датасете, скачанном с сайта госзакупок, регионы обозначены только цифрами. Необходимо соотнести код региона и его наименование с помощью таблицы "regions".

In [ ]:
regions = pd.read_excel('regions.xlsx')
regions.columns = ['region', 'name']
regions['region'] = regions['region'].astype(str)
regions.head()

,region,name
0,1,Россия (код по ОКСМ)
1,2,Центральный федеральный округ
2,3,Белгородская область
3,4,Брянская область
4,5,Владимирская область


In [ ]:
# ТОП-10 регионов по количеству закупок соли за выбранный период
df_regions = df_reg.merge(regions, on='region')
df_regions = df_regions.dropna()
df_regions = df_regions.sort_values('number', ascending=False)
df_regions_10 = df_regions[0:11]
df_regions_10 

,region,number,price,name
12,16,58,8.865172,Тамбовская область
18,24,21,14.033333,Архангельская область
17,23,21,14.404286,Республика Коми
32,42,17,11.701765,Севастополь
4,7,12,15.394167,Ивановская область
29,38,11,12.936364,Краснодарский край
13,17,10,18.000000,Тверская область
26,34,8,9.277500,Южный федеральный округ
0,1,6,15.736111,Россия (код по ОКСМ)
19,25,6,18.270000,Ненецкий автономный округ


In [ ]:
fig = go.Figure(data=[go.Bar(x = df_regions_10['name'], y = df_regions_10['number'], marker_color = 'cornflowerblue')])
                
fig.update_layout(title = 'Распределение закупок по регионам')
fig.update_xaxes({'dtick' : 1}, title = 'регионы')
fig.update_yaxes(title = 'число закупок') 
fig.show()

Можно сделать вывод, что наибольшее количество закупок соли было осуществлено в Тамбовской области.


**5.** Рассмотрим зависимость цены закупки соли от региона.

In [ ]:
df_regions.head()
fig = go.Figure(data=[go.Scatter(x=df_regions['name'], y=df_regions['price'], mode='markers')])

fig.update_layout(title='Средняя цена соли в регионах')
fig.show()

In [ ]:
df_regions['price'].mean()

13.268631174419873

При средней закупочной цене соли 13.27 руб./кг самые высокие средние цены закупки были зафиксировы в Ростовской области (31.38) и Рязанской области (31.23). Самые низкие цены зафиксированы в Республике Карелия (8.25) и Тамбовской области (8,86).



**Выводы**
По результатам исследования было выявлено, что для проведения государственных закупок соли чаще всего применяют электронный аукцион, а также конкурс с ограниченным участием и закупка у единственного поставщика.

Самые крупными заказчиками являются федеральные службы исполнения наказаний и федеральный службы войск национальной гвардии.

В течение рассматриваемого периода большинство закупок соли было проведено в Тамбовской области, далее следуют Архангельская область и Республика Коми.

Цена соли не зависит от сезона, средняя цена за килограмм составляет 13.27 рублей. Самая дорогая соль куплена в Ростовской и Рязанской областях, а самая дешевая - в Республике Карелия и в Тамбовской области.
